In [5]:
import numpy as np
import pandas as pd
import os
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import alpaca_trade_api as tradeapi
%matplotlib inline
import pandas_datareader.data as web


In [6]:
from dotenv import load_dotenv
load_dotenv()
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

api = tradeapi.REST(alpaca_api_key, alpaca_secret_key, api_version='v2') 

In [102]:
assets = api.list_assets()
tradeable = [asset for asset in assets if asset.tradable]

In [35]:
asset_info_df = pd.DataFrame()
asset_info_df['symbol'] = pd.Series([asset.symbol for asset in assets])

In [196]:
main_df = pd.DataFrame()
screened_stocks_df = pd.read_csv('../Data/REIT_stocks.csv')
screened_stocks_tickers = screened_stocks_df['Ticker']
tickers= screened_stocks_tickers.tolist()
tickers.pop(2)

'AIII'

In [197]:
timeframe = '1D'
end_date = datetime.now()
start_date = end_date + timedelta(-365)

for count, ticker in enumerate(tickers):
    df = api.get_barset(
        ticker,
        timeframe,
        limit=None,
        start=start_date,
        end=end_date,
        after=None,
        until=None,
    ).df
    df.index = df.index.date
    series = df[ticker]['close']
    series.rename(ticker, inplace =True)
    if main_df.empty:
        main_df = series
    else:
        main_df = pd.concat([main_df,series], axis = 'columns',join='inner')

        
main_df.head()

,UDR,UMH
2019-12-05,47.81,15.50
2019-12-06,47.88,15.67
2019-12-09,47.84,15.75
2019-12-10,47.73,15.80
2019-12-11,46.80,16.20


# Finding S&P 500 
## finding the close prices of sp500 with pandas DataReader

In [13]:
#THIS is the only thing that worked for me to get closing stock information
sp500 = web.DataReader(['sp500'], 'fred', start_date, end_date)
sp500.head()

,sp500
DATE,
2020-03-31,2584.59
2020-04-01,2470.50
2020-04-02,2526.90
2020-04-03,2488.65
2020-04-06,2663.68


In [61]:
read_stocks = web.DataReader(tickers, 'yahoo',start_date,end_date)
close_stocks = read_stocks['Close']
close_stocks.head()

Symbols,ACC,AHH,AIV,AMH,APTS,AVB,BOWFF,BRG,BRT,CPT,...,NXRT,OPI,RESI,RPT,SNR,SRC,SUI,UDFI,UDR,UMH
Date,,,,,,,,,,,,,,,,,,,,,
2019-05-01,47.110001,16.180000,49.340000,23.879999,15.880000,201.160004,29.010000,11.10,13.99,100.389999,...,38.139999,26.780001,9.93,12.24,5.63,41.029999,123.769997,4.90,44.500000,13.97
2019-05-02,47.150002,15.410000,49.060001,24.010000,15.740000,201.440002,28.870001,11.10,14.00,99.980003,...,37.520000,26.299999,9.94,12.54,5.59,42.400002,123.839996,4.91,44.259998,13.89
2019-05-03,47.310001,16.030001,50.049999,24.280001,16.059999,202.160004,29.010000,11.29,14.05,100.830002,...,38.930000,28.410000,10.29,12.78,6.09,41.700001,123.709999,5.00,44.470001,13.95
2019-05-06,47.110001,16.420000,49.759998,24.160000,16.100000,200.570007,29.010000,11.40,14.19,100.889999,...,39.110001,28.030001,10.07,12.84,6.11,42.150002,123.919998,5.00,44.369999,13.65
2019-05-07,46.250000,16.160000,49.080002,23.660000,15.920000,196.789993,29.010000,11.11,14.18,99.070000,...,38.060001,26.889999,10.03,12.45,6.01,41.509998,121.839996,4.85,43.549999,13.15


In [55]:
sp500_returns = sp500.pct_change()
sp500_returns.dropna(inplace = True)
sp500_returns.head()

,sp500
DATE,
2020-04-01,-0.044142
2020-04-02,0.022829
2020-04-03,-0.015137
2020-04-06,0.070331
2020-04-07,-0.001603


In [58]:
stocks_returns = close_stocks.pct_change()
stocks_returns.dropna(inplace =True)
stocks_returns.head()

Symbols,ACC,AHH,AIV,AMH,APTS,AVB,BOWFF,BRG,BRT,CPT,...,NXRT,OPI,RESI,RPT,SNR,SRC,SUI,UDFI,UDR,UMH
Date,,,,,,,,,,,,,,,,,,,,,
2019-11-13,0.004533,0.012048,0.015524,0.014792,0.002320,0.012441,0.011032,0.006873,-0.009217,0.012620,...,0.017457,0.005979,-0.019964,0.007607,0.003995,0.006475,0.016620,-0.040000,0.014034,-0.013690
2019-11-14,0.005801,0.007937,0.003356,0.008439,0.029321,0.004868,0.000000,0.000000,0.005233,0.006593,...,0.007626,0.001564,0.002778,0.007550,0.002653,0.013671,0.004008,0.031250,0.007549,-0.014541
2019-11-15,0.005554,0.006187,0.007432,0.008368,0.020240,0.008983,0.057800,0.005119,-0.017930,0.007447,...,0.009249,0.000000,-0.000923,-0.007493,0.025132,0.006942,0.010137,-0.006734,0.008117,-0.004024
2019-11-18,0.008286,-0.010062,0.007008,-0.001886,0.005878,0.003449,0.016448,-0.008489,-0.004711,0.002672,...,0.001666,0.009994,0.001848,0.002745,0.015484,0.005318,0.012733,-0.003390,0.005161,-0.006734
2019-11-19,-0.000421,0.002823,0.001648,-0.002645,-0.005113,0.008315,-0.014262,0.000000,0.015976,0.001776,...,-0.006446,-0.005257,0.000000,0.001369,0.010165,0.002743,0.013688,0.020408,0.003902,0.010169


# Beta Section 

In [62]:
#getting covariance for the stocks with sp500
def get_covariance():
    covariance_dict = {}
    for symbol in stocks_returns:
        covariance = stocks_returns[symbol].cov(sp500_returns['sp500'])
        covariance_dict.update({symbol: covariance})
    return covariance_dict
get_covariance()

{'ACC': 0.0011240945276641216,
 'AHH': 0.0009390742316747348,
 'AIV': 0.0013268141713028874,
 'AMH': 0.0009733429717981739,
 'APTS': 0.001073637338452023,
 'AVB': 0.0009710513982013433,
 'BOWFF': 0.0006513669386020639,
 'BRG': 0.0017602711465842023,
 'BRT': 0.000879070865990023,
 'CPT': 0.000921703885997883,
 'ELS': 0.0009692042383082839,
 'EQR': 0.001043342063953411,
 'ESS': 0.0011518202711771751,
 'GADS': -0.0011550799363074054,
 'IIPZF': -7.919820647994424e-05,
 'INVH': 0.0010939480176094737,
 'IRT': 0.0010543409439077252,
 'MAA': 0.0008732743594183367,
 'MRTI': 0.0,
 'NXRT': 0.0010329110525175405,
 'OPI': 0.0011164958749908924,
 'RESI': 0.0003302354801133428,
 'RPT': 0.0011102976268071397,
 'SNR': 0.0018644974551372008,
 'SRC': 0.0011448204895382944,
 'SUI': 0.0010108191826834379,
 'UDFI': 7.599386192675535e-05,
 'UDR': 0.0011995133424716703,
 'UMH': 0.0006967074020868744}

In [66]:
#getting variance 
def get_variance():
    variance_dict = {}
    for symbol in stocks_returns:
        variance = stocks_returns[symbol].var()
        variance_dict.update({symbol: variance})
    return variance_dict
get_variance()


{'ACC': 0.002697257450345539,
 'AHH': 0.0031022386781251955,
 'AIV': 0.0021083782471739212,
 'AMH': 0.001149081111028096,
 'APTS': 0.002420822371392772,
 'AVB': 0.00153491792070373,
 'BOWFF': 0.003604300279023995,
 'BRG': 0.004918647625685296,
 'BRT': 0.0025862222261392185,
 'CPT': 0.0014181562779970478,
 'ELS': 0.001510989335440608,
 'EQR': 0.0013024545091907204,
 'ESS': 0.0014060260788601724,
 'GADS': 0.18507106342953353,
 'IIPZF': 0.0003255217556433757,
 'INVH': 0.001569361874635021,
 'IRT': 0.0022061779256892246,
 'MAA': 0.001422943843677112,
 'MRTI': 0.00036611022455599755,
 'NXRT': 0.0029065478623882045,
 'OPI': 0.002262984439709204,
 'RESI': 0.001371553252782101,
 'RPT': 0.0036170180403495007,
 'SNR': 0.005284164906213356,
 'SRC': 0.0034015778071187722,
 'SUI': 0.0014749517587731172,
 'UDFI': 0.0023827502739682633,
 'UDR': 0.0013498174153744045,
 'UMH': 0.0023027455156116607}

In [178]:
#calculating beta
def get_beta():
    beta_dict = {}
    for symbol in stocks_returns:
        covariance = stocks_returns[symbol].cov(sp500_returns['sp500'])
        variance = stocks_returns[symbol].var()
        beta = covariance/variance
        beta_dict.update({beta:symbol})
    return beta_dict

get_beta()

{0.41675462886203785: 'ACC',
 0.30270856923305917: 'AHH',
 0.6293055684298368: 'AIV',
 0.8470620241310145: 'AMH',
 0.4435010809299185: 'APTS',
 0.6326406025386264: 'AVB',
 0.1807193874474984: 'BOWFF',
 0.35787705900947736: 'BRG',
 0.3399053867471874: 'BRT',
 0.6499311114707781: 'CPT',
 0.6414368490732344: 'ELS',
 0.8010583529721058: 'EQR',
 0.8192026367753612: 'ESS',
 -0.006241277890247852: 'GADS',
 -0.24329620096639434: 'IIPZF',
 0.6970654985892837: 'INVH',
 0.4779038588097295: 'IRT',
 0.6137096437774084: 'MAA',
 0.0: 'MRTI',
 0.3553738322646561: 'NXRT',
 0.4933732001861945: 'OPI',
 0.2407748145713504: 'RESI',
 0.3069649126493865: 'RPT',
 0.3528461901226515: 'SNR',
 0.3365557263286557: 'SRC',
 0.6853235549373151: 'SUI',
 0.03189333886853119: 'UDFI',
 0.8886485896605181: 'UDR',
 0.30255510101463096: 'UMH'}

# Rolling Averages 
## Getting a 20 day and 40 day rolling averages

In [69]:
# Calculating Rolling Average of 20 days
rolling_20 = close_stocks.rolling(window = 20).mean()
rolling_20.dropna(inplace = True)
rolling_20.head()

Symbols,ACC,AHH,AIV,AMH,APTS,AVB,BOWFF,BRG,BRT,CPT,...,NXRT,OPI,RESI,RPT,SNR,SRC,SUI,UDFI,UDR,UMH
Date,,,,,,,,,,,,,,,,,,,,,
2019-12-10,47.4405,17.9475,53.3780,26.4150,13.5845,214.059501,35.9795,12.0240,17.6610,110.9470,...,47.941500,32.894500,11.5960,14.5400,7.7870,51.4865,160.580001,3.0550,47.860999,15.2260
2019-12-11,47.4605,18.0080,53.3135,26.4195,13.6250,214.213501,36.0805,12.0475,17.7010,110.8895,...,47.989999,33.003500,11.6755,14.5530,7.8065,51.5840,160.672501,3.0555,47.849999,15.2690
2019-12-12,47.4225,18.0525,53.1815,26.3860,13.6390,214.092001,36.1410,12.0520,17.7275,110.7110,...,47.934499,33.050999,11.7530,14.5505,7.8080,51.5820,160.485501,3.0595,47.784500,15.3075
2019-12-13,47.3825,18.0870,53.0480,26.3525,13.6235,213.893002,36.1795,12.0480,17.7190,110.4005,...,47.833499,32.994999,11.8265,14.5365,7.8090,51.5345,160.251001,3.0610,47.695000,15.3600
2019-12-16,47.3410,18.1200,52.9055,26.3185,13.6065,213.635001,36.1275,12.0465,17.7230,110.0675,...,47.659499,32.941999,11.8955,14.5365,7.8005,51.4855,159.872501,3.0635,47.589000,15.4045


In [70]:
# Calculating Rolling Average of 40 days
rolling_40 = close_stocks.rolling(window = 40).mean()
rolling_40.dropna(inplace = True)
rolling_40.head()

Symbols,ACC,AHH,AIV,AMH,APTS,AVB,BOWFF,BRG,BRT,CPT,...,NXRT,OPI,RESI,RPT,SNR,SRC,SUI,UDFI,UDR,UMH
Date,,,,,,,,,,,,,,,,,,,,,
2020-01-09,46.74800,18.13050,52.20050,26.13700,13.38800,211.077251,35.70575,11.93450,17.49475,108.270749,...,46.18950,32.40625,11.99200,14.56275,7.72825,50.27525,155.110751,2.96700,47.02325,15.46925
2020-01-10,46.73350,18.15000,52.16800,26.15075,13.38950,211.063750,35.76075,11.94200,17.49650,108.192500,...,46.17625,32.43150,12.02125,14.55875,7.73125,50.27250,154.991750,2.96950,46.99875,15.47750
2020-01-13,46.71875,18.16900,52.12450,26.16425,13.39700,211.049501,35.80650,11.95025,17.50225,108.099249,...,46.15000,32.47400,12.05875,14.55475,7.73650,50.26875,154.890250,2.97025,46.97125,15.49350
2020-01-14,46.69800,18.18350,52.06275,26.16900,13.39725,210.971751,35.85875,11.95575,17.50650,107.991249,...,46.10925,32.51600,12.09025,14.54950,7.74325,50.26050,154.772750,2.96900,46.93075,15.52125
2020-01-15,46.67475,18.19875,52.01075,26.17225,13.39750,210.906001,35.87300,11.96350,17.52100,107.882749,...,46.07475,32.58100,12.12400,14.54900,7.74525,50.25600,154.624501,2.96800,46.89150,15.55325


In [104]:
rolling_100 = close_stocks.rolling(window = 100).mean()
rolling_100.dropna(inplace =True)

# Top 5 for beta

In [195]:
#getting the top pick of the day
beta_dict = get_beta()  
search_key = 1
result = beta_dict.get(search_key) or beta_dict[min(beta_dict, key = lambda key: abs(key-search_key))]    
  
print("The stock best to choose today according to beta: " + str(result))


The stock best to choose today according to beta: UDR


In [194]:
#getting the top 5 picks of the day
sorted_beta_list = sorted(beta_dict, key=lambda key: abs(key), reverse = True)
top_5_beta_list= sorted_beta_list[0:5]
top_5_beta_dict = {}
for keys in top_5_beta_list:
    for values in beta_dict.keys():
        if keys == values:
            top_5_beta_dict.update({beta_dict[values]:keys})
top_5_beta_dict

{'UDR': 0.8886485896605181,
 'AMH': 0.8470620241310145,
 'ESS': 0.8192026367753612,
 'EQR': 0.8010583529721058,
 'INVH': 0.6970654985892837}